# ChainerによるDeep Learning実装（応用）

# Step1 データセットの準備

In [1]:
from sklearn.datasets import load_iris

x, t = load_iris(return_X_y=True)
x = x.astype('float32')
t = t.astype('int32')

In [2]:
# TupleDatasetにて、入力データと目的データをタプルとして扱えるようにする
from chainer.datasets import TupleDataset

dataset = TupleDataset(x, t)

In [4]:
dataset[0]

(array([5.1, 3.5, 1.4, 0.2], dtype=float32), 0)

In [8]:
# データセットの分割
# 訓練データ・検証データ・テストデータに分割

from chainer.datasets import split_dataset_random

# datasetのうち、70%：train_val、30%：test
# train_valのうち、70%：train、30%：valid
train_val, test = split_dataset_random(dataset, int(len(dataset)*0.7), seed=0)
train, valid = split_dataset_random(train_val, int(len(train_val)*0.7), seed=0)

In [11]:
# serialIterator
# イテレータとして、next()をすると次のデータを返してくれる
from chainer.iterators import SerialIterator

train_iter = SerialIterator(train, batch_size=4, repeat=True, shuffle=True)
minibatch = train_iter.next()

minibatch

[(array([5. , 3.6, 1.4, 0.2], dtype=float32), 0),
 (array([6.2, 3.4, 5.4, 2.3], dtype=float32), 2),
 (array([6.1, 3. , 4.6, 1.4], dtype=float32), 1),
 (array([5.3, 3.7, 1.5, 0.2], dtype=float32), 0)]

# Step2　ニューラルネットワークを設定（応用）

In [12]:
import chainer
import chainer.links as L
import chainer.functions as F

In [15]:
# chainer.Chainはlinsクラスを継承しているので、linksを自由に使える
# forwardを定義することで、自由にネットワークを組める（今回は基礎と同じ構成）
class Net(chainer.Chain):
    def __init__(self, n_in=4, n_hidden=3, n_out=3):
        super().__init__()
        with self.init_scope():
            self.l1 = L.Linear(n_in, n_hidden)
            self.l2 = L.Linear(n_hidden, n_hidden)
            self.l3 = L.Linear(n_hidden, n_out)
    
    def forward(self, x):
        h = F.relu(self.l1(x))
        h = F.relu(self.l2(h))
        h = self.l3(h)
        
        return h

In [16]:
# init_scopeを使ったメリットは、n_in/n_hidden/n_outを設定可能

# 隠れそ層の変更
net = Net(n_hidden=100)

# step3 目的関数の設定（応用）

In [18]:
#　目的関数に正則化を追加

# L2ノルム（waight_decay）を適応
from chainer import optimizers
from chainer.optimizer_hooks import WeightDecay

optimizer = optimizers.SGD(lr=0.01)
optimizer.setup(net)

for param in net.params():
    if param.name != 'b':
        param.update_rule.add_hook(WeightDecay(0.0001))

# Step4 最適化手法の設定（応用）

In [19]:
# 最適化手法にMomentumを使う

from chainer import optimizers
from chainer.optimizer_hooks import WeightDecay

optimizer = optimizers.MomentumSGD(lr=0.001, momentum=0.9)
optimizer.setup(net)

for param in net.params():
    if param.name != 'b':
        param.update_rule.add_hook(WeightDecay(0.0001))

# Step5 ニューラルネットワークの訓練（応用）

高速化のためにGPUを用いる ※ローカルでは動かない

In [20]:
gpu_id = 0
n_batch = 64
n_epoch = 50

# ネットワークをGPUメモリに転送
net.to_gpu(gpu_id)

# ログ
results_train, results_valid = {}, {}
results_train['loss'], results_train['accuracy'] = [], []
results_valid['loss'], results_valid['accuracy'] = [], []

train_iter.reset()

count = 1

for epoch in range(n_epoch):
    while True:
        
        # ミニバッチの取得
        train_batch = train_iter.next()
        
        # xとtの分割
        # データをGPUに転送するため、concat_examplesにgpu_idを渡す
        x_train, t_train = chainer.dataset.concat_excamples(train_batch, gpu_id)
        
        # 予測値と目的関数の計算
        y_train = net(x_train)
        loss_train = F.softmax_cross_entropy(y_train, t_train)
        acc_train = F.accuracy(y_train, t_train)
        
        # 勾配の初期化と勾配計算
        net.cleargrads()
        loss_train.backward()
        
        # パラメータの更新
        optimizer.update()
        
        # カウントアップ
        count += 1
        
        # 1エポック終えたら、validデータを取得
        if train_iter.is_new_epoch:
            
            # 訓練データに対する結果の確認
            with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):
                x_valid, t_valid = chainer.dataset.concat_examples(valid, gpu_id)
                y_valid = net(x_valid)
                loss_valid = F.softmax_cross_entropy(y_valid, t_valid)
                acc_valid = F.accuracy(y_valid, t_valid)
                
            # 注意：GPU結果をCPUに転送
            loss_train.to_cpu()
            loss_valid.to_cpu()
            acc_train.to_cpu()
            acc_valid.to_cpu()
            
            # 結果の表示
            print(f"epoch:{epoch}, iteration:{iteration}, \
                    loss(train):{loss_train.array.mean():.4f}, loss(valid):{loss_valid.array.mean():.4f}\
                    acc(train):{acc_train.array.mean():.4f}, acc(valid):{acc_valid.array.mean():.4f}")
            
            # 可視化用に保存
            results_train['loss'] .append(loss_train.array)
            results_train['accuracy'] .append(acc_train.array)
            results_valid['loss'].append(loss_valid.array)
            results_valid['accuracy'].append(acc_valid.array)

            break

RuntimeError: CUDA environment is not correctly set up
(see https://github.com/chainer/chainer#installation).No module named 'cupy'